In [1]:
import pandas as pd
import boxball_loader as bbl
import baseball_stats_utils as bsu

In [2]:
runner_cols = ['base1_run_id', 'base2_run_id', 'base3_run_id']
cols = ['bat_id', 'pit_id', 'bat_hand_cd', 'pit_hand_cd', 'ob_fl', 'tb_ct'] + runner_cols
ev = bbl.load_event_data(bbl.Seasons(1979, 2003), cols, pa_only=True)
ev['baserunner_fl'] = ev[runner_cols].any(axis=1) # a flag indicating whether any runners on
ev.sample(10)

,h_fl,yr,ob_fl,event_cd,game_type,date,pit_hand_cd,pit_id,base1_run_id,game_id,tb_ct,base3_run_id,ab_fl,base2_run_id,bat_event_fl,bat_id,bat_hand_cd,baserunner_fl
10220081,0,1988,0,3,RS,1988-06-03,R,scotm001,None,SFN198806030,0,None,True,None,True,speic001,R,False
8707906,1,1979,1,21,RS,1979-08-14,L,candj001,None,PIT197908140,2,None,True,None,True,dadep101,R,False
191196,1,1998,1,20,RS,1998-06-10,R,nagyc001,None,CLE199806100,1,None,True,None,True,guilj001,R,False
7724579,0,2000,1,14,RS,2000-09-20,L,glavt001,None,ATL200009200,0,None,False,None,True,ventr001,L,False
8540063,0,1985,0,2,RS,1985-06-27,R,guanc001,None,PIT198506270,0,None,True,None,True,palmd001,R,False
4496052,0,1997,0,2,RS,1997-09-06,R,halld002,None,LAN199709060,0,None,True,None,True,cangj001,L,False
11946608,1,1980,1,20,RS,1980-06-04,R,romoe101,None,PIT198006040,1,None,True,None,True,boonb001,R,False
6835379,0,1994,0,2,RS,1994-08-04,R,clemr001,None,BOS199408040,0,molip001,True,None,True,olerj001,L,True
6930965,0,1994,0,2,RS,1994-06-11,R,darlr001,buhnj001,SEA199406110,0,None,True,None,True,blowm001,R,True
13794582,0,1983,0,2,RS,1983-08-02,R,stews001,None,CLE198308022,0,None,True,None,True,tablp001,R,False


In [3]:
runner_ids = ['hendr001']

rickey_fl = ev[runner_cols].isin(runner_ids).any(axis=1)
rickey_fl.value_counts()

False    4121871
True        9839
dtype: int64

In [4]:
# Each player's career line with runners on (will be used for the baseline)
batter_totals = bsu.summarize_events(ev.query('baserunner_fl'), ['bat_id'])
batter_totals.sample(10, random_state=1234)

,pa,ob,ab,h,tb,k,bb,ibb,hr,ba,obp,slg,woba,k%,bb%,hr%
bat_id,,,,,,,,,,,,,,,,
crowd101,3,0,3,0,0,2,0,0,0,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000
sweer101,241,66,219,54,59,26,12,1,0,0.246575,0.273859,0.269406,0.233833,0.107884,0.049793,0.000000
dauer001,1524,456,1340,347,459,73,101,7,15,0.258955,0.299213,0.342537,0.271693,0.047900,0.066273,0.009843
litts001,2,1,2,1,1,1,0,0,0,0.500000,0.500000,0.500000,0.431000,0.500000,0.000000,0.000000
sanda001,11,3,11,3,5,3,0,0,0,0.272727,0.272727,0.454545,0.302364,0.272727,0.000000,0.000000
ankir001,44,9,39,7,12,13,2,0,1,0.179487,0.204545,0.307692,0.211091,0.295455,0.045455,0.022727
offej001,2558,889,2127,599,798,339,288,32,22,0.281617,0.347537,0.375176,0.305259,0.132525,0.112588,0.008600
gatej101,8,2,7,1,3,1,1,0,0,0.142857,0.250000,0.428571,0.288000,0.125000,0.125000,0.000000
hallm001,2156,658,1937,551,840,230,98,57,51,0.284461,0.305195,0.433660,0.293946,0.106679,0.045455,0.023655


In [5]:
counting_cols = ['ob', 'ab', 'h', 'tb', 'k', 'bb', 'ibb', 'hr']
batter_rates = pd.concat([(batter_totals[col]/batter_totals['pa']).rename(col) for col in counting_cols], axis=1)
batter_rates.sample(10, random_state=1234)

,ob,ab,h,tb,k,bb,ibb,hr
bat_id,,,,,,,,
crowd101,0.000000,1.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000
sweer101,0.273859,0.908714,0.224066,0.244813,0.107884,0.049793,0.004149,0.000000
dauer001,0.299213,0.879265,0.227690,0.301181,0.047900,0.066273,0.004593,0.009843
litts001,0.500000,1.000000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000
sanda001,0.272727,1.000000,0.272727,0.454545,0.272727,0.000000,0.000000,0.000000
ankir001,0.204545,0.886364,0.159091,0.272727,0.295455,0.045455,0.000000,0.022727
offej001,0.347537,0.831509,0.234167,0.311962,0.132525,0.112588,0.012510,0.008600
gatej101,0.250000,0.875000,0.125000,0.375000,0.125000,0.125000,0.000000,0.000000
hallm001,0.305195,0.898423,0.255566,0.389610,0.106679,0.045455,0.026438,0.023655


In [6]:
rickey_ct = ev[rickey_fl]['bat_id'].value_counts()
rickey_ct

murpd002    898
mattd001    636
lansc001    479
randw001    411
cansj001    328
           ... 
bergs001      1
willg002      1
youne002      1
noboj001      1
rossd001      1
Name: bat_id, Length: 247, dtype: int64

In [7]:
teammate_baseline = pd.concat([(batter_rates[col]*rickey_ct).dropna().rename(col) for col in batter_rates.columns], axis=1).sum()/rickey_ct.sum()
teammate_baseline['ba'] = teammate_baseline['h'] / teammate_baseline['ab']
teammate_baseline['slg'] = teammate_baseline['tb'] / teammate_baseline['ab']
teammate_baseline['obp'] = teammate_baseline['ob']
teammate_baseline


ob     0.334725
ab     0.851979
h      0.238456
tb     0.368962
k      0.135296
bb     0.090608
ibb    0.017913
hr     0.026353
ba     0.279884
slg    0.433064
obp    0.334725
dtype: float64

In [9]:
with_rickey = bsu.summarize_events(ev, rickey_fl).loc[True]
with_rickey

pa      9839.000000
ob      3327.000000
ab      8190.000000
h       2395.000000
tb      3688.000000
k       1260.000000
bb       881.000000
ibb      181.000000
hr       248.000000
ba         0.292430
obp        0.338144
slg        0.450305
woba       0.317882
k%         0.128062
bb%        0.089542
hr%        0.025206
Name: True, dtype: float64

In [10]:
rate_cols = ['ba', 'obp', 'slg']
comp = pd.concat([teammate_baseline[rate_cols].rename('baseline'), with_rickey[rate_cols+['pa']].rename('with_runner')], axis=1).T
comp['r27'] = 31 * comp['obp'] * comp['slg']
comp

,ba,obp,slg,pa,r27
baseline,0.279884,0.334725,0.433064,NaN,4.49368
with_runner,0.292430,0.338144,0.450305,9839.0,4.72031


In [11]:
# Estimate the size of the difference in runs
(comp['r27']['with_runner']-comp['r27']['baseline'])*comp['pa']['with_runner']/36

61.939209676952565